In [ ]:
import cv2
import numpy as np
import mediapipe as mp
import tensorflow as tf
import time
import nltk
import subprocess
from textblob import TextBlob

# ================== NLTK DATA SETUP ================== #
def download_nltk_data():
    try:
        nltk.data.find('tokenizers/punkt')
        nltk.data.find('corpora/wordnet')
    except LookupError:
        print("Downloading NLTK data...")
        nltk.download('punkt')
        nltk.download('wordnet')
        try:
            subprocess.run(["python", "-m", "textblob.download_corpora"], check=True)
        except:
            print("TextBlob corpora download failed - using fallback correction")

download_nltk_data()

# ================== MODEL SETUP ================== #
# Load your trained model
model = tf.keras.models.load_model("modelAdditionalLayers4001.keras")

# Initialize MediaPipe
mp_hands = mp.solutions.hands
hands = mp_hands.Hands(
    static_image_mode=False,
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
)
mp_drawing = mp.solutions.drawing_utils

# ASL Labels (A-Z)
labels = [chr(i) for i in range(ord('A'), ord('Z') + 1)]

# ================== HAND PROCESSING ================== #
def draw_hand_landmarks_on_black(hand_landmarks, image_size=128):
    """Convert MediaPipe landmarks to a black image with green connections"""
    landmarks = np.array([[lm.x, lm.y] for lm in hand_landmarks.landmark])
    x_vals, y_vals = landmarks[:, 0], landmarks[:, 1]
    
    # Normalize landmarks
    min_x, max_x = np.min(x_vals), np.max(x_vals)
    min_y, max_y = np.min(y_vals), np.max(y_vals)
    width, height = max_x - min_x, max_y - min_y
    scale = 0.7 * image_size / max(width, height)
    
    landmarks[:, 0] = (landmarks[:, 0] - min_x) * scale
    landmarks[:, 1] = (landmarks[:, 1] - min_y) * scale
    landmarks[:, 0] += (image_size - width * scale) / 2
    landmarks[:, 1] += (image_size - height * scale) / 2

    # Draw on black image
    black_img = np.zeros((image_size, image_size, 3), dtype=np.uint8)
    points = landmarks.astype(np.int32)
    
    # Draw connections
    for connection in mp_hands.HAND_CONNECTIONS:
        start = tuple(points[connection[0]])
        end = tuple(points[connection[1]])
        cv2.line(black_img, start, end, (0, 255, 0), 1)
    
    # Draw landmarks
    for point in points:
        cv2.circle(black_img, tuple(point), 1, (0, 255, 0), -1)
    
    return black_img

# ================== AUTO-CORRECTION ================== #
# Common ASL misclassifications
ASL_COMMON_ERRORS = {
    'A': 'S', 'B': 'V', 'D': 'F', 'E': 'M',
    'G': 'Q', 'I': 'J', 'K': 'P', 'M': 'N',
    'S': 'A', 'V': 'B', 'F': 'D', 'N': 'M'
}

def simple_autocorrect(word):
    """Basic letter substitution without NLTK"""
    return ''.join([ASL_COMMON_ERRORS.get(c, c) for c in word])

def advanced_autocorrect(word):
    """Try TextBlob correction with fallback to simple"""
    try:
        tb = TextBlob(word)
        if tb.words:
            suggestion, confidence = tb.words[0].spellcheck()[0]
            return suggestion if confidence > 0.7 else word
    except:
        pass
    return simple_autocorrect(word)

# ================== MAIN LOOP ================== #
# Tracking variables
sentence = ""
current_letter = ""
confirmed_letter = ""
stable_count = 0
last_confirmed_time = time.time()
no_hand_start_time = None

# Thresholds
STABILITY_THRESHOLD = 3
LETTER_COOLDOWN = 1.0
NO_HAND_TIMEOUT = 2.0

cap = cv2.VideoCapture(0)

while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break

    frame = cv2.flip(frame, 1)
    rgb_frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = hands.process(rgb_frame)

    prediction_text = "Show hand sign..."
    current_time = time.time()

    if results.multi_hand_landmarks:
        no_hand_start_time = None
        
        for hand_landmarks in results.multi_hand_landmarks:
            # Process hand image
            hand_img = draw_hand_landmarks_on_black(hand_landmarks)
            gray_img = cv2.cvtColor(hand_img, cv2.COLOR_BGR2GRAY)
            normalized_img = gray_img.astype("float32") / 255.0
            input_img = np.expand_dims(normalized_img, axis=(0, -1))
            
            # Get prediction
            prediction = model.predict(input_img, verbose=0)
            predicted_index = np.argmax(prediction)
            confidence = prediction[0][predicted_index]

            if confidence > 0.8:
                predicted_letter = labels[predicted_index]
                
                # Track stability
                if predicted_letter == confirmed_letter:
                    stable_count += 1
                else:
                    stable_count = 0
                    confirmed_letter = predicted_letter
                
                # Update display
                if stable_count > STABILITY_THRESHOLD:
                    prediction_text = f"Letter: {confirmed_letter}"
                    current_letter = confirmed_letter
                else:
                    prediction_text = f"Detecting: {confirmed_letter}"
                
                # Add to sentence if stable
                if (stable_count > STABILITY_THRESHOLD and 
                    (current_time - last_confirmed_time > LETTER_COOLDOWN)):
                    sentence += confirmed_letter
                    last_confirmed_time = current_time
                    stable_count = 0

            # Draw landmarks
            mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
            cv2.imshow("Hand Input", hand_img)
            break
    else:
        # Handle no hand
        if no_hand_start_time is None:
            no_hand_start_time = current_time
        elif current_time - no_hand_start_time > NO_HAND_TIMEOUT:
            if not sentence.endswith(" "):
                sentence += " "
            no_hand_start_time = None

    # Apply auto-correction
    corrected_sentence = advanced_autocorrect(sentence)

    # Display
    cv2.putText(frame, prediction_text, (10, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (0, 255, 0), 2)
    cv2.putText(frame, f"Raw: {sentence}", (10, 80), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 0, 255), 1)
    cv2.putText(frame, f"Corrected: {corrected_sentence}", (10, 120), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)
    cv2.imshow("ASL Translator", frame)

    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()

TextBlob corpora download failed - using fallback correction


[nltk_data] Downloading package punkt to C:\Users\Denin
[nltk_data]     Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to C:\Users\Denin
[nltk_data]     Thomas\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
